# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [3]:
data = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [4]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [24]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [27]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [28]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed = indexer.fit(data).transform(data)
indexed.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)]

In [29]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [31]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [32]:
assembler = VectorAssembler(
    inputCols=["Age", "Tonnage",
               "passengers",'length', 'cabins', 'passenger_density', 'cruise_cat'],
    outputCol="features")
output = assembler.transform(indexed)

In [33]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [34]:
final_data = output.select("features",'Crew')

In [35]:
final_data.show()

+--------------------+----+
|            features|Crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [36]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [37]:
train_data.describe().show()

+-------+------------------+
|summary|              Crew|
+-------+------------------+
|  count|               122|
|   mean| 7.745737704918045|
| stddev|3.3214317628736714|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [38]:
test_data.describe().show()

+-------+-----------------+
|summary|             Crew|
+-------+-----------------+
|  count|               36|
|   mean|7.958333333333332|
| stddev|4.109105915959265|
|    min|              0.6|
|    max|             21.0|
+-------+-----------------+



In [39]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Crew')
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.008630662814793596,0.011485483834026047,-0.15759358671658713,0.4392573294737695,0.8593472510581202,-0.004257980840676463,0.05485439039643832] Intercept: -1.2127835155386515


In [40]:
test_results = lrModel.evaluate(test_data)
# Interesting results....
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|0.009626132507758456|
| -1.3005627195159661|
| -1.5661918016101488|
| -0.3495856275279987|
|  0.3856151870807327|
|-0.20080278546498498|
|  0.6875417255857137|
|   0.990075856019887|
|   0.990075856019887|
| -0.5306623147345864|
| -0.8426176924590791|
| -0.2620135532129524|
|  0.8678228783392843|
| -1.1915594209882894|
| 0.12721821361514085|
|-0.10787902288131246|
|  0.7714246069291644|
|-0.02436083481202367|
| -0.5157175602624022|
| -1.3978618677331918|
+--------------------+
only showing top 20 rows



In [41]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))

0.7278756973389728

In [42]:
test_results.r2

0.9677258938319139

RMSE=0.7278756973389728 look quiet good for real data

In [47]:
final_data.describe().show()

+-------+-----------------+
|summary|             Crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [48]:
unlabeled_data = test_data.select('features')
predictions = lrModel.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...| 20.99037386749224|
|[5.0,86.0,21.04,9...| 9.300562719515966|
|[5.0,160.0,36.34,...|15.166191801610148|
|[6.0,112.0,38.0,9...|11.249585627527999|
|[8.0,77.499,19.5,...| 8.614384812919267|
|[9.0,59.058,17.0,...| 7.600802785464985|
|[9.0,88.5,21.24,9...| 9.612458274414287|
|[9.0,113.0,26.74,...|11.389924143980114|
|[9.0,113.0,26.74,...|11.389924143980114|
|[10.0,110.0,29.74...|12.130662314734586|
|[11.0,85.0,18.48,...|  8.84261769245908|
|[11.0,91.62700000...| 9.262013553212952|
|[11.0,108.977,26....|11.132177121660716|
|[11.0,138.0,31.14...|13.041559420988289|
|[12.0,2.329,0.94,...|0.4727817863848591|
|[12.0,25.0,3.88,5...|2.9778790228813126|
|[12.0,77.104,20.0...| 8.818575393070835|
|[13.0,25.0,3.82,5...| 2.974360834812024|
|[13.0,61.0,13.8,7...| 6.515717560262402|
|[13.0,63.0,14.4,7...| 6.707861867733191|
+--------------------+------------

In [49]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 0.7278756973389728
MSE: 0.529803030776696


### We can still check which columns used for training are strongly correlated with our results

In [51]:
from pyspark.sql.functions import corr

In [52]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       NULL|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     NULL|       NULL| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

1 means strong correlation:

In [53]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [54]:
data.select(corr('crew', 'cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [55]:
data.select(corr('crew', 'age')).show()

+-------------------+
|    corr(crew, age)|
+-------------------+
|-0.5306565039638852|
+-------------------+



In [56]:
data.select(corr('crew', 'Tonnage')).show()

+-------------------+
|corr(crew, Tonnage)|
+-------------------+
|  0.927568811544939|
+-------------------+



In [57]:
data.select(corr('crew', 'length')).show()

+------------------+
|corr(crew, length)|
+------------------+
|0.8958566271016579|
+------------------+

